<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci-551-lab-assignments-hw/blob/main/homework-5/solution/PySpark_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!pip install pyspark
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=1aacc411c7d28f0ca6e4c8024be19c576fe141c4ee3c2d6baf1af7e6b27bab91
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.8 MB/s eta 0:00:00
time: 546 µs (started: 2023-04-20 08:54:09 +00:00)


In [ ]:
%%bash
java --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
time: 373 ms (started: 2023-04-20 08:54:09 +00:00)


# Spark Session

In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

time: 12.8 s (started: 2023-04-20 08:54:09 +00:00)


In [ ]:
import pyspark.sql.functions as fc

time: 676 µs (started: 2023-04-20 08:58:03 +00:00)


# Assignment Solution

Load Data to dataframes

In [ ]:
country_df = spark.read.json("country.json")
city_df = spark.read.json("city.json")
country_lang_df = spark.read.json("countrylanguage.json")

aqi_df = spark.read.csv("aqi.csv", header=True, inferSchema=True)

time: 1.85 s (started: 2023-04-20 09:06:24 +00:00)


### Sample View

In [ ]:
country_df.show(5)

+-------+----+-----+-------------+------+------+--------------------+--------------------+---------+--------------+--------------------+-----------+----------+--------------------+-----------+
|Capital|Code|Code2|    Continent|   GNP|GNPOld|      GovernmentForm|         HeadOfState|IndepYear|LifeExpectancy|           LocalName|       Name|Population|              Region|SurfaceArea|
+-------+----+-----+-------------+------+------+--------------------+--------------------+---------+--------------+--------------------+-----------+----------+--------------------+-----------+
|    129| ABW|   AW|North America| 828.0| 793.0|Nonmetropolitan T...|             Beatrix|        0|          78.4|               Aruba|      Aruba|    103000|           Caribbean|      193.0|
|      1| AFG|   AF|         Asia|5976.0|   0.0|     Islamic Emirate|       Mohammad Omar|     1919|          45.9|Afganistan/Afqane...|Afghanistan|  22720000|Southern and Cent...|   652090.0|
|     56| AGO|   AO|       Africa|6

In [ ]:
city_df.show(5)

+-----------+-------------+---+--------------+----------+
|CountryCode|     District| ID|          Name|Population|
+-----------+-------------+---+--------------+----------+
|        AFG|        Kabol|  1|         Kabul|   1780000|
|        AFG|     Qandahar|  2|      Qandahar|    237500|
|        AFG|        Herat|  3|         Herat|    186800|
|        AFG|        Balkh|  4|Mazar-e-Sharif|    127800|
|        NLD|Noord-Holland|  5|     Amsterdam|    731200|
+-----------+-------------+---+--------------+----------+
only showing top 5 rows

time: 330 ms (started: 2023-04-20 09:07:22 +00:00)


In [ ]:
country_lang_df.show(5)

+-----------+----------+----------+----------+
|CountryCode|IsOfficial|  Language|Percentage|
+-----------+----------+----------+----------+
|        ABW|         T|     Dutch|       5.3|
|        ABW|         F|   English|       9.5|
|        ABW|         F|Papiamento|      76.7|
|        ABW|         F|   Spanish|       7.4|
|        AFG|         F|   Balochi|       0.9|
+-----------+----------+----------+----------+
only showing top 5 rows

time: 317 ms (started: 2023-04-20 09:07:38 +00:00)


In [ ]:
aqi_df.show(5)

+----------+---------+--------------------+-----+
|      date|  country|              status|value|
+----------+---------+--------------------+-----+
|2022-07-21|  Albania|                Good|   14|
|2022-07-21|  Algeria|            Moderate|   65|
|2022-07-21|  Andorra|            Moderate|   55|
|2022-07-21|   Angola|Unhealthy for Sen...|  113|
|2022-07-21|Argentina|            Moderate|   63|
+----------+---------+--------------------+-----+
only showing top 5 rows

time: 317 ms (started: 2023-04-20 09:07:53 +00:00)


## Question 2

### a.i

In [102]:
joined_df = aqi_df.join(country_df, aqi_df.country == country_df.Name)
joined_df.show(10)

+----------+----------+--------------------+-----+----+-----+-------+----+-----+-------------+--------+--------+--------------------+--------------------+---------+--------------+--------------------+----------+----------+--------------------+-----------+
|      date|   country|              status|value|year|month|Capital|Code|Code2|    Continent|     GNP|  GNPOld|      GovernmentForm|         HeadOfState|IndepYear|LifeExpectancy|           LocalName|      Name|Population|              Region|SurfaceArea|
+----------+----------+--------------------+-----+----+-----+-------+----+-----+-------------+--------+--------+--------------------+--------------------+---------+--------------+--------------------+----------+----------+--------------------+-----------+
|2022-07-21|   Albania|                Good|   14|2022|    7|     34| ALB|   AL|       Europe|  3205.0|  2500.0|            Republic|      Rexhep Mejdani|     1912|          71.6|          ShqipÃ«ria|   Albania|   3401200|     South

In [121]:
result_df = joined_df.select("Name").distinct().sort("Name")
result_df.show(truncate=False)

+----------------------+
|Name                  |
+----------------------+
|Albania               |
|Algeria               |
|Andorra               |
|Angola                |
|Argentina             |
|Armenia               |
|Australia             |
|Austria               |
|Azerbaijan            |
|Bahrain               |
|Bangladesh            |
|Belarus               |
|Belgium               |
|Belize                |
|Bermuda               |
|Bolivia               |
|Bosnia and Herzegovina|
|Brazil                |
|Brunei                |
|Bulgaria              |
+----------------------+
only showing top 20 rows

time: 541 ms (started: 2023-04-21 00:19:06 +00:00)


### a.ii

In [133]:
country_set = country_df.select("Name").distinct()
aqi_set = aqi_df.select("country").distinct()

result_set = country_set.intersect(aqi_set).sort("Name")

result_set.show(truncate=False)

+----------------------+
|Name                  |
+----------------------+
|Albania               |
|Algeria               |
|Andorra               |
|Angola                |
|Argentina             |
|Armenia               |
|Australia             |
|Austria               |
|Azerbaijan            |
|Bahrain               |
|Bangladesh            |
|Belarus               |
|Belgium               |
|Belize                |
|Bermuda               |
|Bolivia               |
|Bosnia and Herzegovina|
|Brazil                |
|Brunei                |
|Bulgaria              |
+----------------------+
only showing top 20 rows

time: 612 ms (started: 2023-04-21 00:23:43 +00:00)


### b.i

In [134]:
joined_df = aqi_df.join(country_df, aqi_df.country == country_df.Name, "left_anti")

result_df = joined_df.select("country").distinct().sort("country")
result_df.show(truncate=False)

+----------------------------------------------------+
|country                                             |
+----------------------------------------------------+
|Ivory Coast                                         |
|Jersey                                              |
|Kazakhstan                                          |
|Kosovo                                              |
|Montenegro                                          |
|Palestinian Territory                               |
|Reunion                                             |
|Russia                                              |
|Serbia                                              |
|United Kingdom of Great Britain and Northern Ireland|
|United States of America                            |
|Vatican                                             |
+----------------------------------------------------+

time: 602 ms (started: 2023-04-21 00:23:48 +00:00)


### b.ii

In [135]:
country_set = country_df.select("Name").distinct()
aqi_set = aqi_df.select("country").distinct()

result_set = aqi_set.subtract(country_set).sort("country")

result_set.show(truncate=False)

+----------------------------------------------------+
|country                                             |
+----------------------------------------------------+
|Ivory Coast                                         |
|Jersey                                              |
|Kazakhstan                                          |
|Kosovo                                              |
|Montenegro                                          |
|Palestinian Territory                               |
|Reunion                                             |
|Russia                                              |
|Serbia                                              |
|United Kingdom of Great Britain and Northern Ireland|
|United States of America                            |
|Vatican                                             |
+----------------------------------------------------+

time: 833 ms (started: 2023-04-21 00:23:54 +00:00)


### c.i

In [136]:
joined_df = country_df.join(aqi_df, aqi_df.country == country_df.Name, "left_anti")

result_df = joined_df.select("Name").distinct().sort("Name")
result_df.show(truncate=False)

+-------------------------------------+
|Name                                 |
+-------------------------------------+
|Afghanistan                          |
|American Samoa                       |
|Anguilla                             |
|Antarctica                           |
|Antigua and Barbuda                  |
|Aruba                                |
|Bahamas                              |
|Barbados                             |
|Benin                                |
|Bhutan                               |
|Botswana                             |
|Bouvet Island                        |
|British Indian Ocean Territory       |
|Burundi                              |
|Cameroon                             |
|Christmas Island                     |
|Cocos (Keeling) Islands              |
|Comoros                              |
|Congo                                |
|Congo, The Democratic Republic of the|
+-------------------------------------+
only showing top 20 rows

time: 727 ms (

### c.ii

In [137]:
country_set = country_df.select("Name").distinct()
aqi_set = aqi_df.select("country").distinct()

result_set = country_set.subtract(aqi_set).sort("Name")

result_set.show(truncate=False)

+-------------------------------------+
|Name                                 |
+-------------------------------------+
|Afghanistan                          |
|American Samoa                       |
|Anguilla                             |
|Antarctica                           |
|Antigua and Barbuda                  |
|Aruba                                |
|Bahamas                              |
|Barbados                             |
|Benin                                |
|Bhutan                               |
|Botswana                             |
|Bouvet Island                        |
|British Indian Ocean Territory       |
|Burundi                              |
|Cameroon                             |
|Christmas Island                     |
|Cocos (Keeling) Islands              |
|Comoros                              |
|Congo                                |
|Congo, The Democratic Republic of the|
+-------------------------------------+
only showing top 20 rows

time: 659 ms (

### d

In [138]:
aqi_df = aqi_df.withColumn('year', fc.year('date'))
aqi_df = aqi_df.withColumn('month', fc.month('date'))

aqi_aug_2022_df = aqi_df.filter(
    (aqi_df['year'] == 2022) & (aqi_df['month'] == 8)
)

aqi_aug_2022_df = aqi_aug_2022_df.groupBy('status')

aqi_aug_2022_df = aqi_aug_2022_df.agg(fc.avg('value').alias('avg_value'))

aqi_aug_2022_df = aqi_aug_2022_df.filter(fc.count('value') >= 100)
aqi_aug_2022_df = aqi_aug_2022_df.sort("avg_value")
aqi_aug_2022_df.show(truncate=False)

+------------------------------+------------------+
|status                        |avg_value         |
+------------------------------+------------------+
|Good                          |27.929097605893187|
|Moderate                      |71.38070175438597 |
|Unhealthy for Sensitive Groups|122.953125        |
|Unhealthy                     |167.9704433497537 |
+------------------------------+------------------+

time: 557 ms (started: 2023-04-21 00:24:13 +00:00)


## Question 3

In [80]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

time: 5.04 ms (started: 2023-04-20 23:47:29 +00:00)


Convert Spark Dataframe to RDD

In [57]:
country_rdd = country_df.rdd
city_rdd = city_df.rdd
country_lang_rdd = country_lang_df.rdd

aqi_rdd = aqi_df.rdd

time: 718 µs (started: 2023-04-20 23:22:02 +00:00)


Sample

In [58]:
aqi_rdd.take(5)

[Row(date=datetime.date(2022, 7, 21), country='Albania', status='Good', value=14, year=2022, month=7),
 Row(date=datetime.date(2022, 7, 21), country='Algeria', status='Moderate', value=65, year=2022, month=7),
 Row(date=datetime.date(2022, 7, 21), country='Andorra', status='Moderate', value=55, year=2022, month=7),
 Row(date=datetime.date(2022, 7, 21), country='Angola', status='Unhealthy for Sensitive Groups', value=113, year=2022, month=7),
 Row(date=datetime.date(2022, 7, 21), country='Argentina', status='Moderate', value=63, year=2022, month=7)]

time: 85.9 ms (started: 2023-04-20 23:22:05 +00:00)


In [59]:
country_rdd.take(5)

[Row(Capital=129, Code='ABW', Code2='AW', Continent='North America', GNP=828.0, GNPOld=793.0, GovernmentForm='Nonmetropolitan Territory of The Netherlands', HeadOfState='Beatrix', IndepYear=0, LifeExpectancy=78.4, LocalName='Aruba', Name='Aruba', Population=103000, Region='Caribbean', SurfaceArea=193.0),
 Row(Capital=1, Code='AFG', Code2='AF', Continent='Asia', GNP=5976.0, GNPOld=0.0, GovernmentForm='Islamic Emirate', HeadOfState='Mohammad Omar', IndepYear=1919, LifeExpectancy=45.9, LocalName='Afganistan/Afqanestan', Name='Afghanistan', Population=22720000, Region='Southern and Central Asia', SurfaceArea=652090.0),
 Row(Capital=56, Code='AGO', Code2='AO', Continent='Africa', GNP=6648.0, GNPOld=7984.0, GovernmentForm='Republic', HeadOfState='JosÃ© Eduardo dos Santos', IndepYear=1975, LifeExpectancy=38.3, LocalName='Angola', Name='Angola', Population=12878000, Region='Central Africa', SurfaceArea=1246700.0),
 Row(Capital=62, Code='AIA', Code2='AI', Continent='North America', GNP=63.2, GN

time: 96.5 ms (started: 2023-04-20 23:22:07 +00:00)


### a.i

In [146]:
country_key_rdd = country_rdd.map(lambda x: (x["Name"], x)).distinct()

aqi_key_rdd = aqi_rdd.map(lambda x: (x["country"], x)).distinct()

country_aqi_join_rdd = country_key_rdd.join(aqi_key_rdd).map(lambda x: x[0]).distinct()
country_aqi_join_rdd = country_aqi_join_rdd.sortBy(lambda x: x)

country_aqi_join_rdd.collect()

['Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Belize',
 'Bermuda',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Cambodia',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Ethiopia',
 'Finland',
 'France',
 'French Guiana',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Grenada',
 'Guadeloupe',
 'Guam',
 'Guatemala',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Jordan',
 'Kenya',
 'Kuwait',
 'Kyrgyzstan',
 'Laos',
 'Latvia',
 'Lebanon',
 'Liberia',
 'Liechtenstein',
 'Lithuania',
 'Luxem

time: 1.84 s (started: 2023-04-21 00:29:47 +00:00)


In [153]:
country_key_rdd = country_rdd.map(lambda x: x["Name"]).distinct()

aqi_key_rdd = aqi_rdd.map(lambda x: x["country"]).distinct()

country_aqi_join_rdd = country_key_rdd.intersection(aqi_key_rdd).distinct()
country_aqi_join_rdd = country_aqi_join_rdd.sortBy(lambda x: x)

country_aqi_join_rdd.collect()

['Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Belize',
 'Bermuda',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Cambodia',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Ethiopia',
 'Finland',
 'France',
 'French Guiana',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Grenada',
 'Guadeloupe',
 'Guam',
 'Guatemala',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Jordan',
 'Kenya',
 'Kuwait',
 'Kyrgyzstan',
 'Laos',
 'Latvia',
 'Lebanon',
 'Liberia',
 'Liechtenstein',
 'Lithuania',
 'Luxem

time: 979 ms (started: 2023-04-21 00:32:24 +00:00)
